In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Day1").getOrCreate()

data = [
    ("2025-01-01", 100),
    ("2025-01-02", 150),
    ("2025-01-04", 120),
    ("2025-01-06", 200),
]

columns = ["date", "sales"]
sales_df = spark.createDataFrame(data, columns).withColumn("date", to_date(col("date")))

min_max = sales_df.agg(min("date").alias("min_date"), max("date").alias("max_date")).collect()[0]
min_date, max_date = min_max["min_date"], min_max["max_date"]

date_df = spark.sql(f"SELECT sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day) as all_dates")
date_df = date_df.select(explode(col("all_dates")).alias("date"))

missing_df = date_df.join(sales_df, on="date", how="left_anti")
missing_df.show()


+----------+
|      date|
+----------+
|2025-01-05|
|2025-01-03|
+----------+



In [ ]:
"""
WITH ranked_logins AS (
    SELECT
        user_id,
        login_date,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY login_date) AS rn
    FROM user_logins
),
grouped_logins AS (
    SELECT
        user_id,
        login_date,
        DATE_SUB(login_date, rn) AS grp_key
    FROM ranked_logins
)
SELECT
    user_id,
    MAX(streak) AS longest_streak
FROM (
    SELECT
        user_id,
        grp_key,
        COUNT(*) AS streak
    FROM grouped_logins
    GROUP BY user_id, grp_key
) t
GROUP BY user_id;

"""